In [1]:
# !ls $FLINK_HOME/lib

In [2]:
from pyflink.table import DataTypes, TableEnvironment, EnvironmentSettings
from pyflink.table.expressions import lit

In [3]:
settings = EnvironmentSettings.new_instance().in_batch_mode().use_blink_planner().build()
t_env = TableEnvironment.create(settings)

In [4]:
content = "line Licensed to the Apache Software Foundation ASF under one " \
              "line or more contributor license agreements See the NOTICE file " \
              "line distributed with this work for additional information " \
              "line regarding copyright ownership The ASF licenses this file " \
              "to you under the Apache License Version the " \
              "License you may not use this file except in compliance " \
              "with the License"

with open('test.txt', 'w') as fopen:
    fopen.write('\n'.join(content.split()))

In [5]:
hdfs_path = 'hdfs://hdfs:9000/user/word_count_hudi'

my_source_ddl = """
    create table mySource (
        word VARCHAR
    ) with (
        'connector' = 'filesystem',
        'format' = 'csv',
        'path' = 'test.txt'
    )
"""

my_sink_ddl = f"""
    create table mySink (
        word VARCHAR PRIMARY KEY NOT ENFORCED,
        `count` BIGINT
    ) with (
        'connector' = 'hudi',
        'path' = '{hdfs_path}',
        'table.type' = 'MERGE_ON_READ'
    )
"""

t_env.execute_sql(my_source_ddl)
t_env.execute_sql(my_sink_ddl)

In [6]:
tab = t_env.from_path('mySource')
tab

In [7]:
tab.to_pandas().head()

,word
0,line
1,Licensed
2,to
3,the
4,Apache


In [8]:
tab.group_by(tab.word) \
   .select(tab.word, lit(1).count) \
   .execute_insert('mySink', overwrite = True).wait()

In [9]:
tab = t_env.from_path('mySink')
tab.to_pandas()

2022-04-16 14:19:10,423 INFO  org.apache.hadoop.conf.Configuration.deprecation             [] - mapred.job.map.memory.mb is deprecated. Instead, use mapreduce.map.memory.mb


,word,count
0,copyright,1
1,or,1
2,may,1
3,in,1
4,line,4
5,use,1
6,more,1
7,work,1
8,one,1
9,License,3
